We would like to give awards to the best wineries. Come up with 3 relevant ones. Which wineries should we choose and why? Be creative ;)

In [61]:
# Import required libraries
import sqlite3
  
# Connect to SQLite database
conn = sqlite3.connect(r'../data/vivino.db')

# Create a cursor object
cur = conn.cursor()

In [126]:
cur.execute("""
            SELECT *
            FROM wineries 
            WHERE wineries.id = 1574
            ;
            """)
cur.fetchall()

[]

In [128]:

cur.execute("""
            SELECT wines.name, wines.winery_id, wineries.id, wineries.name
            FROM wines
                join wineries on wines.winery_id = wineries.id
            LIMIT 15;
            """)
cur.fetchall()

[('The Armagh Shiraz', 1651, 1651, "Tenuta Tignanello 'Solaia'"),
 ('Colheita Porto', 1652, 1652, 'Tignanello'),
 ('Lion Tamer Napa Valley Red Blend', 1471, 1471, 'Siepi'),
 ('Raut Lagrein', 75712, 75712, 'Corte di Cama Sforzato di Valtellina')]

In [77]:

cur.execute("""
            SELECT  w.name, count(v.id)
            FROM wines w
            JOIN vintages v ON v.wine_id = w.id
            GROUP BY w.name
            LIMIT 10;
            """)
cur.fetchall()


[("'Giulio Ferrari' Riserva del Fondatore", 10),
 ("'Giulio Ferrari' Riserva del Fondatore Rosé", 1),
 ('110 e Lode Merlot', 1),
 ('123 Brunello di Montalcino Riserva', 1),
 ('1694 Classification Red', 1),
 ('25 anni Montefalco Sagrantino', 1),
 ('30 Year Old Tawny Port', 1),
 ('40 Anos Tawny Port', 1),
 ('40 Years Old Port', 1),
 ('40 Years Old Tawny Port', 1)]

In [139]:
cur.execute("""
            SELECT tl.country_code, tl.name,vtr.rank 
            FROM toplists tl
            JOIN vintage_toplists_rankings vtr ON tl.id = vtr.top_list_id
            LIMIT 15;
            """)
cur.fetchall()

[('global', "Vivino's 2016 Wine Style Awards: French Champagne", 7),
 ('global', "Vivino's 2018 Wine Style Awards: French Champagne", 3),
 ('global', "Vivino's 2019 Wine Style Awards: French Champagne Vintage", 3),
 ('global', "Vivino's 2020 Wine Style Awards: French Champagne Vintage", 1),
 ('global', "Vivino's 2019 Wine Style Awards: Bordeaux Pauillac", 2),
 ('global', "Vivino's 2020 Wine Style Awards: Bordeaux Pauillac", 2),
 ('global', "Vivino's 2018 Wine Style Awards: Bordeaux Pauillac", 2),
 ('global', "Vivino's 2020 Wine Style Awards: Tuscan Red", 1),
 ('global', "Vivino's 2016 Wine Style Awards: Central Italy Red", 1),
 ('global', "Vivino's 2019 Wine Style Awards: Tuscan Red", 2),
 ('global', "Vivino's 2016 Wine Style Awards: French Champagne", 6),
 ('global',
  "Vivino's 2019 Wine Style Awards: Napa Valley Cabernet Sauvignon",
  1),
 ('global', "Vivino's 2018 Wine Style Awards: Bordeaux Pauillac", 5),
 ('global', "Vivino's 2019 Wine Style Awards: Bordeaux Pauillac", 6),
 ('glo

In [ ]:
cur.execute("""
            SELECT  DISTINCT country_code
            FROM toplists
            LIMIT 15;
            """)
cur.fetchall()